### Question 1. Spark version
Install Spark and PySpark
- Install Spark
- Run PySpark
- Create a local spark session
- Execute spark.version.

In [111]:
# IMPORTS
import os

import pandas as pd
import pyspark
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, IntegerType, StringType, StructField, StructType, TimestampType

In [112]:
# SPARK SESSION
spark = (SparkSession
         .builder
         .master("local[*]")
         .appName("test")
         .getOrCreate())

display(f"Spark Version : {spark.version}")

'Spark Version : 3.5.1'

### Question 2: FHV October 2019
Read the October 2019 FHV into a Spark Dataframe with a schema as we did in the lessons.

Repartition the Dataframe to 6 partitions and save it to parquet.

What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.

In [113]:
# SCHEMA
fhv_schema = StructType([
    StructField("dispatching_base_num", StringType(), True),
    StructField("pickup_datetime", TimestampType(), True),
    StructField("dropOff_datetime", TimestampType(), True),
    StructField("PUlocationID", IntegerType(), True),
    StructField("DOlocationID", IntegerType(), True),
    StructField("SR_Flag", DoubleType(), True),
    StructField("Affiliated_base_number", StringType(), True)
])

In [114]:
csv_path = "./data/raw/fhv/fhv_tripdata_2019-10.csv.gz"
pq_dir = "./data/pq/fhv/"

# READ CSV
fhv_df = (spark.read
          .option("header", "true")
          .schema(fhv_schema)
          .csv(csv_path))

# REPARTITION TO PARQUET
fhv_df.repartition(6).write.parquet(pq_dir, mode="overwrite")

# AVERAGE SIZE OF THE PARQUET FILES
pq_file_sizes = [os.path.getsize(os.path.join(pq_dir, f)) for f in os.listdir("./data/pq/fhv/") if f[-8:] == ".parquet"]
average_pq_size = round(sum(pq_file_sizes) / len(pq_file_sizes) / 1024 / 1024)

display(f"Average file size of partitioned parquet is: {average_pq_size} MB")


'Average file size of partitioned parquet is: 6 MB'

### Question 3: Count records
How many taxi trips were there on the 15th of October?

Consider only trips that started on the 15th of October.

In [115]:
# CREATE TEMP VIEW FOR QUERY
fhv_df.createOrReplaceTempView("fhv_data")

## Using spark.sql
# spark.sql("""
# SELECT count(1) as trips_count FROM fhv_data WHERE CAST(pickup_datetime AS DATE) = '2019-10-15';
# """).show()

## Using spark.filter
number_of_trips = fhv_df.filter(F.to_date("pickup_datetime") == '2019-10-15').count()

display(f"Number of trips on 2019-10-15: {number_of_trips:,d}")

'Number of trips on 2019-10-15: 62,610'

### Question 4: Longest trip for each day
What is the length of the longest trip in the dataset in hours?

In [116]:
max_duration = spark.sql("""
SELECT
    MAX(UNIX_TIMESTAMP(dropOff_datetime) - UNIX_TIMESTAMP(pickup_datetime)) / 3600 AS trip_duration
FROM fhv_data;
""").collect()

display(f"The longest trip in the dataset took : {max_duration[0]['trip_duration']:,.1f} Hrs")

'The longest trip in the dataset took : 631,152.5 Hrs'

### Question 5: User Interface

Spark’s User Interface which shows the application's dashboard runs on which local port?

[Link to Spark dashboard / localhost:4040](http://localhost:4040)

### Question 6: Least frequent pickup location zone
Load the zone lookup data into a temp view in Spark

Using the zone lookup data and the FHV October 2019 data, what is the name of the LEAST frequent pickup location Zone?

In [117]:
# CREATE `zone_data` TEMP VIEW
(spark.read
 .option("header", "true")
 .schema(StructType([
    StructField("LocationID", IntegerType(), True),
    StructField("Borough", StringType(), True),
    StructField("Zone", StringType(), True),
    StructField("service_zone", StringType(), True),]))
 .csv("./zones/taxi_zone_lookup.csv")
 .createOrReplaceTempView("zone_data"))

In [118]:
# JOIN `fhv_data` and `zone_data`
least_picked_up_zone = spark.sql("""
SELECT
    pul.Zone as pickup_zone,
    COUNT(1) as picked_up_count
FROM 
    fhv_data
        LEFT JOIN zone_data pul ON fhv_data.PUlocationID = pul.LocationID
        LEFT JOIN zone_data dol ON fhv_data.DOlocationID = dol.LocationID
GROUP BY 
    pickup_zone
ORDER BY
    picked_up_count
LIMIT 1;
""").collect()

display(f"The least picked up zone in the dataset is : {least_picked_up_zone[0]['pickup_zone']}")

'The least picked up zone in the dataset is : Jamaica Bay'